In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\forjb\AppData\Local\Temp\ipykernel_4188\2736925689.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Forest Web Scraping for Assignmnet3

In [4]:
# imports
import requests                 # How Python gets the webpages
from bs4 import BeautifulSoup   # Creates structured, searchable object
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from time import sleep
from datetime import datetime
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10
rcParams['font.size'] = 20
rcParams['axes.facecolor'] = 'white'
%matplotlib inline

### Get response from website

In [9]:
url = "http://www.ufcstats.com/statistics/fighters?char=a&page=all"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)

# Always check if the request was successful
if response.status_code != 200:
    print(f"Error: Received status code {response.status_code}")
    exit()

print(f"Response status: {response.status_code}")

Response status: 200


### Get page content
The 3 cells below gets the reponse from the fighters URl. I.E the page that contains the table of all fighters.
Take note that this only gets fighters whose Last Name starts with 'a' to change this you would need to change the char=a in the Url used.

The code then gets the all the data in the table for each fighter and places it into the relevant column in the dataframe.

In [12]:
page = response.content

In [14]:
scraping = BeautifulSoup(page, "lxml") 
scraping

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
    Stats | UFC
  </title>
<meta content="" name="description"/>
<meta content="" name="viewport"/>
<link href="/blocks/main.css?ver=96342" rel="stylesheet"/>
<script src="/js/vendor/modernizr-2.6.2.min.js"></script>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 'auto');
    ga('send', 'pa

In [18]:
def create_fighter_dataframe(soup):
    # Define the column names based on the table headers
    columns = ['First', 'Last', 'Nickname', 'HT', 'WT', 'Reach', 'Stance', 'W', 'L', 'D', 'Belt']
    
    # Initialize an empty list to store fighter data
    fighters_data = []
    
    # Find the table containing fighter data
    table = soup.find('table', {'class': 'b-statistics__table'})
    
    # Find all table rows in the tbody
    rows = table.find('tbody').find_all('tr', {'class': 'b-statistics__table-row'})
    
    for row in rows:
        # Skip empty separator rows
        if row.find('td', {'class': 'b-statistics__table-col_type_clear'}):
            continue
            
        # Extract all columns from the row
        cols = row.find_all('td', {'class': 'b-statistics__table-col'})
        
        # For each column, get the text and clean it up
        fighter_data = []
        for i, col in enumerate(cols):
            text = col.get_text(strip=True)
            
            # Handle special cases
            if i == 3 or i == 5:  # HT or Reach columns with '--'
                text = None if text == '--' else text
            elif i == 4:  # WT column - remove 'lbs.' and convert to number
                text = None if text == '--' else text.replace(' lbs.', '')
            elif i in (7, 8, 9):  # W, L, D columns - convert to integers
                text = int(text) if text else 0
                
            fighter_data.append(text)
        
        # Only add if we have all expected columns
        if len(fighter_data) == len(columns):
            fighters_data.append(fighter_data)
    
    df = pd.DataFrame(fighters_data, columns=columns)
    
    return df

df = create_fighter_dataframe(scraping)
df

,First,Last,Nickname,HT,WT,Reach,Stance,W,L,D,Belt
0,Tom,Aaron,,None,155,None,,5,3,0,
1,Danny,Abbadi,The Assassin,"5' 11""",155,None,Orthodox,4,6,0,
2,Nariman,Abbasov,Bayraktar,"5' 8""",155,"66.0""",Orthodox,28,4,0,
3,David,Abbott,Tank,"6' 0""",265,None,Switch,10,15,0,
4,Hamdy,Abdelwahab,The Hammer,"6' 2""",264,"72.0""",Southpaw,6,0,0,
...,...,...,...,...,...,...,...,...,...,...,...
221,Abu,Azaitar,Captain Morocco,"5' 9""",185,"76.0""",Orthodox,14,4,1,
222,Ottman,Azaitar,Bulldozer,"5' 8""",155,"71.0""",Switch,13,3,0,
223,Luiz,Azeredo,,"5' 9""",154,None,Orthodox,15,10,0,
224,Luciano,Azevedo,,"6' 3""",161,None,Orthodox,17,9,1,


The 2 cells below just gets reponse from 1 fighters URL and uses beautiful soup on it.

In [20]:
url2 = "http://www.ufcstats.com/fighter-details/15df64c02b6b0fde"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
response2 = requests.get(url2, headers=headers)

# Always check if the request was successful
if response2.status_code != 200:
    print(f"Error: Received status code {response2.status_code}")
    exit()

print(f"Response status: {response2.status_code}")

Response status: 200


In [22]:
page2 = response2.content
scraping2 = BeautifulSoup(page2, "lxml") 
scraping2

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
    Stats | UFC
  </title>
<meta content="" name="description"/>
<meta content="" name="viewport"/>
<link href="/blocks/main.css?ver=445885" rel="stylesheet"/>
<script src="/js/vendor/modernizr-2.6.2.min.js"></script>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 'auto');
    ga('send', 'p

# Section 2 Code made by Karolina and Forest for scraping fighter details


In [36]:
url_ranking = "http://ufcstats.com/statistics/fighters?page=all"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
response_ranking = requests.get(url_ranking, headers=headers)

# Always check if the request was successful
if response_ranking.status_code != 200:
    print(f"Error: Received status code {response_ranking.status_code}")
    exit()

print(f"Response status: {response_ranking.status_code}")

Response status: 200


In [39]:
# I don't think we will need this table, but we can keep it to show the process of getting the data.
fighters_page = response_ranking.content
scraping = BeautifulSoup(fighters_page, "lxml")
tables_fighters = scraping.find_all('table')
tables_fighters_df = pd.read_html(str(tables_fighters))
fighters_table = tables_fighters_df[0]
fighters_table

C:\Users\forjb\AppData\Local\Temp\ipykernel_4628\1706655041.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_fighters_df = pd.read_html(str(tables_fighters))


,First,Last,Nickname,Ht.,Wt.,Reach,Stance,W,L,D,Belt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5.0,3.0,0.0,NaN
2,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4.0,6.0,0.0,NaN
3,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28.0,4.0,0.0,NaN
4,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10.0,15.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
222,Abu,Azaitar,Captain Morocco,"5' 9""",185 lbs.,"76.0""",Orthodox,14.0,4.0,1.0,NaN
223,Ottman,Azaitar,Bulldozer,"5' 8""",155 lbs.,"71.0""",Switch,13.0,3.0,0.0,NaN
224,Luiz,Azeredo,NaN,"5' 9""",154 lbs.,--,Orthodox,15.0,10.0,0.0,NaN
225,Luciano,Azevedo,NaN,"6' 3""",161 lbs.,--,Orthodox,17.0,9.0,1.0,NaN


In [43]:
fighters_table.to_csv('fighters_table.csv', index=False)

In [59]:
url = "https://api.sportradar.com/mma/trial/v2/en/rankings.json"
headers = {
    "accept": "application/json",
    "x-api-key": "h1uj1ZQXspnpdnug4j2jisxL554RwvNL07Zc5F3c"
}
response = requests.get(url, headers=headers)
response

<Response [200]>

In [61]:
res = json.loads(response.content)
rankings = res.get('rankings',[])

records = []
for cat_ranking in rankings:
    category = cat_ranking.get('name')
    year = cat_ranking.get('year')
    week = cat_ranking.get('week')
    competitor_rankings = cat_ranking.get('competitor_rankings', [])
    for competitor in competitor_rankings:
        rank = competitor.get('rank')
        comp = competitor.get('competitor')
        comp_id = comp.get('id')
        name = comp.get('name')
        gender = comp.get('gender')
        records.append({
            'Name': name,
            'Id': comp_id,
            'Ranking': rank,
            "Gender": gender,
            'Category': category,
            'Year': year,
            "Week": week
        })
df_rankings_fighters = pd.DataFrame(records)
df_rankings_fighters.sort_values(by='Id', ascending=False)
df_rankings_fighters

,Name,Id,Ranking,Gender,Category,Year,Week
0,"Makhachev, Islam",sr:competitor:251835,1,male,pound_for_pound,2025,19
1,"Jones, Jon",sr:competitor:253371,2,male,pound_for_pound,2025,19
2,"Topuria, Ilia",sr:competitor:750503,3,male,pound_for_pound,2025,19
3,"Dvalishvili, Merab",sr:competitor:399183,4,male,pound_for_pound,2025,19
4,"Du Plessis, Dricus",sr:competitor:400461,5,male,pound_for_pound,2025,19
...,...,...,...,...,...,...,...
186,"Cavalcanti, Jacqueline",sr:competitor:1049265,11,female,womens_bantamweight,2025,19
187,"Cornolle, Nora",sr:competitor:1027333,12,female,womens_bantamweight,2025,19
188,"Tate, Miesha",sr:competitor:246049,13,female,womens_bantamweight,2025,19
189,"Edwards, Joselyne",sr:competitor:768194,14,female,womens_bantamweight,2025,19


In [65]:
df_rankings_fighters.to_csv('df_ranking_fighters.csv', index=False)

In [67]:
profiles = []
for index, fighter in df_rankings_fighters.iterrows():
    fighterId = fighter['Id'].replace(":", "%3A")
    profile_url = "https://api.sportradar.com/mma/trial/v2/en/competitors/" + fighterId + "/profile.json"
    response_fighter = requests.get(profile_url, headers=headers)
    res_fighter = json.loads(response_fighter.content)
    competitor_profile = res_fighter.get('competitor')
    if competitor_profile is not None:
        profile_id = competitor_profile.get('id')
        info = res_fighter.get('info')
        country = info.get('birth_country')
        Birth_country_code = info.get('birth_country_code')
        DOB = info.get('birth_date')
        reach = info.get('reach')
        Height = info.get('height')
        Weight = info.get('weight')
        Nickname = info.get('nickname')
        record = res_fighter.get('record')
        Wins = record.get('wins')
        Draws = record.get('draws')
        Losses = record.get('losses')
        profiles.append({
            'Id': profile_id,
            'Country': country,
            'Brith_Code': Birth_country_code,
            'DOB': DOB,
            'Reach': reach,
            'Height': Height,
            'Weight': Weight,
            'Nickname': Nickname,
            'Wins': Wins,
            'Draws': Draws,
            'Losses': Losses
        })
df_profiles_fighters = pd.DataFrame(profiles)
df_profiles_fighters

,Id,Country,Brith_Code,DOB,Reach,Height,Weight,Nickname,Wins,Draws,Losses
0,sr:competitor:251835,RUSSIAN FEDERATION,RUS,1991-09-27,179,178,70.3,None,27,0,2
1,sr:competitor:253371,UNITED STATES,USA,1987-07-19,215,194,112.5,Bones,28,0,1
2,sr:competitor:750503,GERMANY,DEU,1997-01-21,175,170,65.5,El Matador,16,0,0
3,sr:competitor:399183,GEORGIA,GEO,1991-01-10,173,168,60.8,The Machine,18,0,4
4,sr:competitor:400461,None,None,1994-01-14,193,185,83.9,Stillknocks,23,0,2
...,...,...,...,...,...,...,...,...,...,...,...
158,sr:competitor:542083,RUSSIAN FEDERATION,RUS,1989-11-11,174,173,61.7,None,15,0,8
159,sr:competitor:1049265,BRAZIL,BRA,1997-08-29,178,175,61.2,None,9,0,1
160,sr:competitor:1027333,None,None,1989-06-12,170,169,62.6,None,9,0,2
161,sr:competitor:246049,UNITED STATES,USA,1986-08-18,165,168,61.5,Cupcake,20,0,10


In [69]:
df_profiles_fighters.to_csv('df_profiles_fighters.csv', index=False)

In [71]:
fighters_df = pd.merge(df_rankings_fighters, df_profiles_fighters, on='Id', how='left')
fighters_df_info = fighters_df.drop_duplicates(subset=['Id', 'Category'])
fighters_df_info = fighters_df_info.rename(columns={'Year': 'Year_Ranking', "Week": 'Week_ranking'})
fighters_df_info

,Name,Id,Ranking,Gender,Category,Year_Ranking,Week_ranking,Country,Brith_Code,DOB,Reach,Height,Weight,Nickname,Wins,Draws,Losses
0,"Makhachev, Islam",sr:competitor:251835,1,male,pound_for_pound,2025,19,RUSSIAN FEDERATION,RUS,1991-09-27,179.0,178.0,70.3,None,27.0,0.0,2.0
2,"Jones, Jon",sr:competitor:253371,2,male,pound_for_pound,2025,19,UNITED STATES,USA,1987-07-19,215.0,194.0,112.5,Bones,28.0,0.0,1.0
4,"Topuria, Ilia",sr:competitor:750503,3,male,pound_for_pound,2025,19,GERMANY,DEU,1997-01-21,175.0,170.0,65.5,El Matador,16.0,0.0,0.0
6,"Dvalishvili, Merab",sr:competitor:399183,4,male,pound_for_pound,2025,19,GEORGIA,GEO,1991-01-10,173.0,168.0,60.8,The Machine,18.0,0.0,4.0
8,"Du Plessis, Dricus",sr:competitor:400461,5,male,pound_for_pound,2025,19,None,None,1994-01-14,193.0,185.0,83.9,Stillknocks,23.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,"Cavalcanti, Jacqueline",sr:competitor:1049265,11,female,womens_bantamweight,2025,19,BRAZIL,BRA,1997-08-29,178.0,175.0,61.2,None,9.0,0.0,1.0
214,"Cornolle, Nora",sr:competitor:1027333,12,female,womens_bantamweight,2025,19,None,None,1989-06-12,170.0,169.0,62.6,None,9.0,0.0,2.0
215,"Tate, Miesha",sr:competitor:246049,13,female,womens_bantamweight,2025,19,UNITED STATES,USA,1986-08-18,165.0,168.0,61.5,Cupcake,20.0,0.0,10.0
216,"Edwards, Joselyne",sr:competitor:768194,14,female,womens_bantamweight,2025,19,PANAMA,PAN,1995-09-29,178.0,173.0,61.7,La Pantera,15.0,0.0,6.0


In [73]:
fighters_df_info.to_csv('Fighters_info.csv', index=False)

### Updated code to find the additional stats
The code below adds stats to each fighter. 
The stats can be found on ufcstats website by clicking on a fighter.
The site will take you to a URL of that fighter and in the top it gives all the basic info plus addtional stats with explanations.
Some of the fighters could not be found on the site. You can see their names printed out as the code runs below.

For this code to work you will need to implement all the code that Karolina made from above see Section 2 above. Remember you will need to have an account for the one API site that karolina found.

In [81]:
import time
import re
df = pd.read_csv('Fighters_info.csv')

# Initialize new columns for the stats
stats_columns = ['SLpM', 'Str_Acc', 'SApM', 'Str_Def', 'TD_Avg', 'TD_Acc', 'TD_Def', 'Sub_Avg']
for col in stats_columns:
    if col not in df.columns:
        df[col] = None

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def get_fighter_urls_for_letter(letter):
    """Get all fighter URLs for a specific starting letter"""
    url = f"http://ufcstats.com/statistics/fighters?char={letter}&page=all"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error fetching {letter}: Status {response.status_code}")
            return {}
        
        soup = BeautifulSoup(response.content, 'html.parser')
        fighters = {}
        
        # Find all fighter links in the table
        table = soup.find('table', {'class': 'b-statistics__table'})
        if table:
            rows = table.find_all('tr', {'class': 'b-statistics__table-row'})
            for row in rows:
                cols = row.find_all('td', {'class': 'b-statistics__table-col'})
                if len(cols) >= 2:  # At least first and last name columns
                    first_name = cols[0].get_text(strip=True)
                    last_name = cols[1].get_text(strip=True)
                    link = cols[0].find('a')
                    if link and 'href' in link.attrs:
                        name_key = f"{last_name}, {first_name}".lower()
                        fighters[name_key] = link['href']
        return fighters
    except Exception as e:
        print(f"Error processing letter {letter}: {str(e)}")
        return {}

def get_fighter_stats(url):
    """Get detailed stats from a fighter's page"""
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Error fetching fighter stats: Status {response.status_code}")
            return None
        
        soup = BeautifulSoup(response.content, 'html.parser')
        stats = {}
        
        # Find career statistics section
        career_stats = soup.find('div', {'class': 'b-list__info-box_style_middle-width'})
        if career_stats:
            # Left box stats
            left_box = career_stats.find('div', {'class': 'b-list__info-box-left'})
            if left_box:
                for item in left_box.find_all('li', {'class': 'b-list__box-list-item_type_block'}):
                    title = item.find('i').get_text(strip=True).replace(':', '')
                    # Get all text and remove the title to get the value
                    value = item.get_text(strip=True).replace(title, '').strip()
                    
                    #Clean the values
                    value = re.sub(r'[^\d.%]', '', value)
                    
                    if 'SLpM' in title:
                        stats['SLpM'] = float(value) if value.replace('.', '').isdigit() else None
                    elif 'Str. Acc' in title:
                        stats['Str_Acc'] = float(value.replace('%', '')) if '%' in value else None
                    elif 'SApM' in title:
                        stats['SApM'] = float(value) if value.replace('.', '').isdigit() else None
                    elif 'Str. Def' in title:
                        stats['Str_Def'] = float(value.replace('%', '')) if '%' in value else None
            
            # Right box stats
            right_box = career_stats.find('div', {'class': 'b-list__info-box-right'})
            if right_box:
                for item in right_box.find_all('li', {'class': 'b-list__box-list-item_type_block'}):
                    title = item.find('i').get_text(strip=True).replace(':', '')
                    # Get all text and remove the title to get the value
                    value = item.get_text(strip=True).replace(title, '').strip()
                    
                    
                    value = re.sub(r'[^\d.%]', '', value)
                    
                    if 'TD Avg' in title:
                        stats['TD_Avg'] = float(value) if value.replace('.', '').isdigit() else None
                    elif 'TD Acc' in title:
                        stats['TD_Acc'] = float(value.replace('%', '')) if '%' in value else None
                    elif 'TD Def' in title:
                        stats['TD_Def'] = float(value.replace('%', '')) if '%' in value else None
                    elif 'Sub. Avg' in title:
                        stats['Sub_Avg'] = float(value) if value.replace('.', '').isdigit() else None
        
        return stats
    except Exception as e:
        print(f"Error getting stats from {url}: {str(e)}")
        return None

#Build a complete fighter URL dictionary for all letters
all_fighter_urls = {}
for letter in 'abcdefghijklmnopqrstuvwxyz':
    fighter_urls = get_fighter_urls_for_letter(letter)
    all_fighter_urls.update(fighter_urls)
    time.sleep(1)  # Be polite with requests

#Match fighters from CSV with their URLs and get stats
for idx, row in df.iterrows():
    # Create lookup key in format "Last, First"
    name_parts = row['Name'].split(', ')
    if len(name_parts) == 2:
        lookup_key = row['Name'].lower()
    else:
        # Handle names without comma (shouldn't happen in your CSV)
        lookup_key = f"{row['Name']}, ".lower()
    
    if lookup_key in all_fighter_urls:
        fighter_stats = get_fighter_stats(all_fighter_urls[lookup_key])
        
        if fighter_stats:
            for stat in stats_columns:
                if stat in fighter_stats:
                    df.at[idx, stat] = fighter_stats[stat]
        
        time.sleep(1)  # Be polite with requests
    else:
        print(f"Could not find URL for {row['Name']}")

# Save the updated DataFrame
df.to_csv('Fighters_info_updated2.csv', index=False)
print("CSV file updated with new stats!")

Could not find URL for Volkanovski, Alex
Could not find URL for Temirov, Ramazonbek
Could not find URL for Song, Yadong
Could not find URL for Volkanovski, Alex
Could not find URL for Hooker, Daniel
Could not find URL for Saint-Denis, Benoit
Could not find URL for Neal, Geoffrey
Could not find URL for Costa, Paulo Henrique
Could not find URL for Magomedov, Abusupiyan
Could not find URL for Rountree, Khalil
Could not find URL for Spivak, Sergey
Could not find URL for Parkin, Michael
Could not find URL for Zhang, Weili
Could not find URL for Yan, Xiaonan
Could not find URL for Lucindo, Jasmin
Could not find URL for Godinez, Lupita
Could not find URL for Peña, Julianna
Could not find URL for Dumont Viana, Norma
CSV file updated with new stats!


In [6]:
data = pd.read_csv('./csv/df_all_fights.csv')
data

,date,competition,winner,winner_abbreviation,winner_gender,winner_qualifier,winner_id,winner_control,winner_knockdowns,winner_significant_strike_percentage,...,loser_takedowns,loser_takedowns_attempted,loser_total_strike_percentage,loser_total_strikes,loser_total_strikes_attempted,final_round,final_round_length,method,scheduled_length,weight_class
0,2019-03-16,UFC Fight Night - London,"Grundy, Mike",GRU,male,home,sr:competitor:539717,0:00,1,52.38,...,0,1,23.36,25,107,2,4:42,ko_tko,3,featherweight(136-145)
1,2019-03-16,UFC Fight Night - London,"McCann, Molly",MCC,female,home,sr:competitor:441632,0:00,0,54.31,...,0,0,32.35,77,238,3,5:00,decision_unanimous,3,flyweight(116-125)
2,2019-03-16,UFC Fight Night - London,"Ige, Dan",IGE,male,away,sr:competitor:539711,0:00,0,62.50,...,1,1,54.55,6,11,1,1:17,submission,3,featherweight(136-145)
3,2019-03-16,UFC Fight Night - London,"Safarov, Saparbek",SAF,male,away,sr:competitor:304962,0:00,0,81.40,...,0,0,55.56,20,36,3,5:00,decision_unanimous,3,light_heavyweight(186-205)
4,2019-03-16,UFC Fight Night - London,"Diakiese, Marc",DIA,male,home,sr:competitor:290246,0:00,1,59.43,...,1,3,33.80,24,71,3,5:00,decision_unanimous,3,lightweight(146-155)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,2025-05-17,UFC Fight Night: Burns vs. Morales,"Delvalle, Yadier",DEL,male,home,sr:competitor:1191035,01:21,0,75.00,...,0,0,30.00,3,10,1,2:54,submission,3,featherweight(136-145)
1541,2025-05-17,UFC Fight Night: Burns vs. Morales,"Gordon, Jared",GOR,male,home,sr:competitor:333581,02:36,1,53.85,...,0,0,68.75,11,16,1,3:37,ko_tko,3,lightweight(146-155)
1542,2025-05-17,UFC Fight Night: Burns vs. Morales,"Green, Gabriel",GRE,male,home,sr:competitor:658401,02:12,0,44.44,...,2,3,68.75,33,48,2,3:43,submission,3,lightweight(146-155)
1543,2025-05-17,UFC Fight Night: Burns vs. Morales,"Costa, Melquizael",COS,male,away,sr:competitor:980481,00:23,0,50.00,...,3,10,42.08,85,202,3,5:00,decision_unanimous,3,featherweight(136-145)


In [14]:
data['strike_accuracy_diff '] = data['winner_significant_strike_percentage'] - data['loser_significant_strike_percentage']
data['total_strike_accuracy_diff  '] = data['winner_total_strike_percentage'] - data['loser_total_strike_percentage']
data['knockdowns_diff'] = data['winner_knockdowns'] - data['loser_knockdowns']
data['takedown_accuracy_diff'] = data['winner_takedown_percentage'] - data['loser_takedown_percentage']
data['takedowns_diff'] = data['winner_takedowns'] - data['loser_takedowns']
data['submission_attempts_diff'] = data['winner_submission_attempts'] - data['loser_submission_attempts']
data.to_csv('./csv/df_all_fights.csv', index=False)